In [ ]:
from Models import Models
import Common
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import datetime

%load_ext tensorboard

## Load Model

In [ ]:
model = Models.create_2DCNN_MLP()

In [ ]:
model = Models.create_3DCNN()

In [ ]:
model = Models.create_LSTM()

In [ ]:
model = Models.create_Transfer_LSTM((40, 128, 128, 3))

## Training

In [ ]:
train = Common.evaluate_dataset("train/")
X_train = train.path
y_train = tf.keras.utils.to_categorical(LabelEncoder().fit_transform(train.category), 51)

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
for i in range(200):
    model.fit(Common.create_batch(X_train, y_train),
              max_queue_size = 5,
              callbacks=[tensorboard_callback] )

In [ ]:
test = Common.evaluate_dataset("train/")
X_test = test.path
y_test = tf.keras.utils.to_categorical(LabelEncoder().fit_transform(test.category), 51)
model.evaluate(Common.create_batch(X_test, y_test), max_queue_size = 5)

In [ ]:
model.save("models/" + model.name)

In [ ]:
%tensorboard --logdir logs/fit --port 6006 --host localhost